In [1]:
# System imports
from datetime import datetime
import numpy as np
import pandas as pd

# Third-party imports
import yfinance as yf
from pathlib import Path

# Local imports
from portfolio.holdings import Portfolio
from portfolio.commsec import Trades
import datehandler

# Vars
today = datetime.today().date()

In [2]:
trades = Trades()
trades_df = trades.all

p_dates = datehandler.date_list(trades_df.index[-1], today)  # Portfolio dates index

tickers = list(sorted(set(trades_df.Ticker.to_list())))
props = ['TradeType','TradeVolume','TradePrice','HoldingVolume','Vwap']

columns = pd.MultiIndex.from_product([tickers,props], names=['Tickers','Props'])
df_p = pd.DataFrame(None, index=pd.DatetimeIndex(p_dates), columns=columns)  # Create multiindex portfolio df

# Clean index and columns
df_p.index.name = 'Date'
df_p = df_p.reindex(sorted(df_p.columns),axis=1)  # Sort columns
df_p = df_p.reindex(sorted(df_p.index),axis=0)  # Sort index
df_p

Tickers              A2M                                                 ACW  \
Props      HoldingVolume TradePrice TradeType TradeVolume Vwap HoldingVolume   
Date                                                                           
2015-01-07           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-08           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-09           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-12           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-13           NaN        NaN       NaN         NaN  NaN           NaN   
...                  ...        ...       ...         ...  ...           ...   
2020-01-20           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-21           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-22           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-23           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-24           NaN        NaN       NaN         NaN  NaN           NaN   

Tickers                                           ...           WZR  \
Props      TradePrice TradeType TradeVolume Vwap  ... HoldingVolume   
Date                                              ...                 
2015-01-07        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-08        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-09        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-12        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-13        NaN       NaN         NaN  NaN  ...           NaN   
...               ...       ...         ...  ...  ...           ...   
2020-01-20        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-21        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-22        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-23        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-24        NaN       NaN         NaN  NaN  ...           NaN   

Tickers                                                    ZNO             \
Props      TradePrice TradeType TradeVolume Vwap HoldingVolume TradePrice   
Date                                                                        
2015-01-07        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-08        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-09        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-12        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-13        NaN       NaN         NaN  NaN           NaN        NaN   
...               ...       ...         ...  ...           ...        ...   
2020-01-20        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-21        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-22        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-23        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-24        NaN       NaN         NaN  NaN           NaN        NaN   

Tickers                                
Props      TradeType TradeVolume Vwap  
Date                                   
2015-01-07       NaN         NaN  NaN  
2015-01-08       NaN         NaN  NaN  
2015-01-09       NaN         NaN  NaN  
2015-01-12       NaN         NaN  NaN  
2015-01-13       NaN         NaN  NaN  
...              ...         ...  ...  
2020-01-20       NaN         NaN  NaN  
2020-01-21       NaN         NaN  NaN  
2020-01-22       NaN         NaN  NaN  
2020-01-23       NaN         NaN  NaN  
2020-01-24       NaN         NaN  NaN  

[1318 rows x 175 columns]

In [3]:
dropped_cols = ['Market','Ticker','TradePrice','Brokerage']
for ticker in tickers:
    # Add trade dataframe values to portfolio dataframe
    df_t = trades_df[trades_df.Ticker == ticker].copy()
    df_t.loc[df_t['TradeType'] == 'S', 'Volume'] *= -1  # Don't worry about the warning
    df_t = df_t.drop(columns=dropped_cols)  # Drop columns first
    df_t = df_t.rename(columns={            # Then rename columns - due to conflicting name!
        'Volume':'TradeVolume',
        'EffectivePrice':'TradePrice',
        })
    df_p[ticker] = df_t

In [4]:
# Lookup prices
# Build list of tickers for yfinance
lookup_tickers = [f'{ticker}.AX' for ticker in df_p.columns.levels[0].to_list()]
lookup_tickers = ' '.join(lookup_tickers)

prices = yf.download(lookup_tickers, start=p_dates[0], end=p_dates[-1])

[*********************100%***********************]  35 of 35 completed

2 Failed downloads:
- SRS.AX: No data found, symbol may be delisted
- IBXO.AX: No data found for this date range, symbol may be delisted


In [5]:
tickers_with_prices = []

for ticker in prices.columns.levels[1]:
    if prices['Close',ticker].isna().sum() > 0.9 * prices.index.isna().sum():
        prices.drop(columns=ticker, level=1)
        tickers_with_prices.append(ticker)

# Remove market suffix from data
tickers_without_suffix = [ticker.replace('.AX','') for ticker in prices.columns.levels[1]]
tickers_with_prices = [ticker.replace('.AX','') for ticker in tickers_with_prices]

prices.columns.set_levels(tickers_without_suffix, level=1, inplace=True)

In [6]:
def _build_from_trades(df):
    df_ticker = df.copy()

    df_ticker['HoldingVolume'] = df_ticker['TradeVolume'].fillna(0).cumsum()

    # Vwap: BookValue / HoldingVolume
    df_ticker.loc[df_ticker['TradeType'] == 'B','TradeEncoding'] = 1
    df_ticker.loc[df_ticker['TradeType'] == 'S','TradeEncoding'] = 0
    df_ticker['BuyValue'] = df_ticker['TradeVolume'] * df_ticker['TradePrice'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['TradeVolume'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['BuyVolume'].cumsum()  # Temp calc column
    df_ticker['BuyValue'] = df_ticker['BuyValue'].cumsum()  # Temp calc column
    df_ticker['Vwap'] = df_ticker['BuyValue'].divide(df_ticker['BuyVolume']).fillna(method='ffill')

    df_ticker = df_ticker.drop(columns=['BuyVolume','BuyValue','TradeEncoding'])  # Remove temp calc columns
    
    return df_ticker

def _dividends_stocksplits(df):
    try:  # Dividends and stock splits
        stock = yf.Ticker(f'{ticker}.AX')
        df_actions = stock.actions
    except TypeError:
        print(f'\rSomething went wrong!',flush=True)
        return
    
    # Add dividends column
    if len(df_actions[df_actions['Dividends'] > 0]) > 0:
        df['Dividends'] = df_actions['Dividends'].replace(0,np.nan).copy()
    
    # Add stocksplits column
    if len(df_actions[df_actions['Stock Splits'] > 0]) > 0:
        df['StockSplits'] = df_actions['Stock Splits'].replace(0,np.nan).copy()

        first_index = df['HoldingVolume'].first_valid_index()
        stocksplit_cumulative = df[df['HoldingVolume'].first_valid_index():]['StockSplits'].cumprod()
        try:
            stocksplit_final_ratio = stocksplit_cumulative[stocksplit_cumulative.last_valid_index()]
            last_index = stocksplit_cumulative.last_valid_index()
        except:
            return
        else:
            if stocksplit_final_ratio:
                print(f'\n{stocksplit_final_ratio}\n', df[first_index]['HoldingVolume'], last_index)
        
        # Multiply HVol / Divide Vwap by the final cumprod of the stocksplits
        # df['HoldingVolume'] = df['HoldingVolume'] * df['StockSplits']
        # df['Vwap'] = df['Vwap'].divide(df['StockSplits'])

    return df


# Build portfolio
for count, ticker in enumerate(df_p.columns.levels[0]):
    print(f'\r{ticker} | Progress {count+1}/{len(df_p.columns.levels[0])} ',end='',flush=True)
    # if ticker != 'QBE':
    if ticker not in tickers_with_prices:
        print(f'Skipping {ticker} due to no price data...')
        continue
    
    df_p[ticker] = _build_from_trades(df_p[ticker])     # Build porfolio from trades
    df_p[ticker, 'ClosePrice'] = prices['Close'][ticker].copy()  # Load prices into dataframe
    df_p[ticker] = _dividends_stocksplits(df_p[ticker])  # Update dataframe with dividends and stocksplits

    # Calculating returns
    df_p[ticker, 'Cashflow'] = df_p[ticker, 'TradePrice'] * df_p[ticker, 'TradeVolume']
    df_p[ticker, 'Cashflow'] = df_p[ticker, 'Cashflow'].fillna(0)
    
    df_p[ticker, 'EndValue'] = df_p[ticker, 'HoldingVolume'] * df_p[ticker, 'ClosePrice'].fillna(method='ffill')
    df_p[ticker, 'StartValue'] = df_p[ticker, 'EndValue'].shift(1).replace(0,np.nan)
    df_p[ticker, 'DailyReturn'] = df_p[ticker, 'EndValue'].divide(
        df_p[ticker, 'StartValue'].add(df_p[ticker, 'Cashflow'])
        ) - 1
    df_p[ticker, 'DailyReturn'] = df_p[ticker, 'DailyReturn'].replace(-1,np.nan) + 1
    df_p[ticker, 'DailyReturn'] = df_p[ticker, 'DailyReturn'].cumprod()

    
# df_p = df_p.drop(['EndValue','StartValue','Cashflow'], axis='columns',level=1)
df_p[('portfolio', 'EndValue')] = df_p.xs('EndValue', level='Props', axis=1).sum(axis=1, min_count=1)
df_p[('portfolio', 'StartValue')] = df_p.xs('StartValue', level='Props', axis=1).sum(axis=1, min_count=1)
df_p[('portfolio', 'Cashflow')] = df_p.xs('Cashflow', level='Props', axis=1).sum(axis=1, min_count=1)
df_p[('portfolio','DailyReturn')] = ((df_p[('portfolio','EndValue')] -
                                        (df_p[('portfolio','StartValue')]+df_p[('portfolio','Cashflow')])
                                    )
                                    /
                                    (df_p[('portfolio','StartValue')]+df_p[('portfolio','Cashflow')])
                                    ) + 1
df_p[('portfolio','DailyReturn')] = df_p[('portfolio','DailyReturn')].cumprod()

ACW | Progress 2/35Skipping ACW due to no price data...
AVZ | Progress 5/35C:\Users\BobJin\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
EHE | Progress 8/35Skipping EHE due to no price data...
FTT | Progress 9/35Skipping FTT due to no price data...
IBXO | Progress 10/35- IBXO.AX: 1d data not available for startTime=-2208988800 and endTime=1579929660. Only 100 years worth of day granularity data are allowed to be fetched per request.
Something went wrong!
IFN | Progress 11/35Skipping IFN due to no price data...
IMM | Progress 12/35Skipping IMM due to no price data...
MDR | Progress 15/35C:\Users\BobJin\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying

KeyError: Timestamp('2015-01-07 00:00:00')

In [0]:
ticker = 'MDR'
# df_p[df_p[ticker,'HoldingVolume'].first_valid_index():df_p[ticker,'HoldingVolume'].last_valid_index()][ticker]
df_p[df_p[ticker,'HoldingVolume'].first_valid_index():][ticker]

In [0]:
test = df_p['RMD'].DailyReturn + 1
test = test.cumprod()
# test[test.isna() == False]
test

In [0]:
'''
The problem is: when I sell out of a position, then buy back in, the vwap is reset.
'''

df_p.RMD.Vwap.unique()

In [0]:
df_p.RMD[df_p.RMD.Vwap == 10.330584795321638]

In [0]:
df_p.RMD[df_p.RMD.TradeType == 'S']

In [0]:
df_p.loc['2018-06-18':'2018-06-25'].RMD